# Computing statistics that can assist in creating an investment portfolio

We are interested in creating a 1-year portfolio as these futures expire within the near future and can't be held for too long.

In [1]:
import numpy as np
import pandas as pd
import yfinance as yf

In [2]:
PATH = 'processed_data/full_dataset.csv'

In [3]:
risk_free_stock = yf.Ticker('^IRX')
risk_free_stock = risk_free_stock.history(period='5y')
risk_free_stock.reset_index(inplace=True)
risk_free_stock['Date'] = risk_free_stock['Date'].dt.date
risk_free_stock = risk_free_stock[['Date', 'Close']]

In [4]:
stock_information = pd.read_csv(PATH)
separate_dfs = {}
for column in stock_information.columns:
    if column not in ['Date', 'diff', 'Prime']:
        separate_dfs[column] = stock_information[['Date', column]].rename(columns={column: 'Close'})
        separate_dfs[column] = separate_dfs[column].iloc[-252:]
    

In [5]:
risk_free_stock.dtypes

Date      object
Close    float64
dtype: object

In [6]:
# calculate the daily returns for each stock
for key in separate_dfs:
    separate_dfs[key]['Daily Return'] = separate_dfs[key]['Close'].pct_change()
    separate_dfs[key].dropna(inplace=True)
    separate_dfs[key]['Date'] = pd.to_datetime(separate_dfs[key]['Date'])

# calculate the daily returns for the risk free stock
risk_free_stock['Daily Return'] = risk_free_stock['Close'].pct_change()
risk_free_stock.dropna(inplace=True)
risk_free_stock['Date'] = pd.to_datetime(risk_free_stock['Date'])

In [7]:
separate_dfs['Maize Jul'].dtypes

Date            datetime64[ns]
Close                  float64
Daily Return           float64
dtype: object

## Sharpe Ratio

In [8]:
# subtract daily returns of risk free stock from daily returns of each stock, match the dates
sharpe_dfs = {}
for key in separate_dfs:
    sharpe_dfs[key] = pd.merge(separate_dfs[key], risk_free_stock, on='Date', how='left')
    sharpe_dfs[key]['Excess Return'] = sharpe_dfs[key]['Daily Return_x'] - sharpe_dfs[key]['Daily Return_y']

In [9]:
for key in sharpe_dfs:
    print(f'Sharpe Ratio for {key}: {sharpe_dfs[key]["Excess Return"].mean() / sharpe_dfs[key]["Excess Return"].std() * np.sqrt(len(sharpe_dfs[key]))}')

Sharpe Ratio for Maize Mar: -1.349088299903418
Sharpe Ratio for Maize May: -1.384392300292323
Sharpe Ratio for Maize Jul: -1.4205865377067177
Sharpe Ratio for Maize Sep: -1.3248755469378672
Sharpe Ratio for Oats Mar: -0.9746272060767236
Sharpe Ratio for Oats May: -1.0280654460066754
Sharpe Ratio for Oats Jul: -0.9057002136514066
Sharpe Ratio for Oats Sep: -0.9231922954440658
Sharpe Ratio for Soybeans Mar: -1.5696991164446301
Sharpe Ratio for Soybeans May: -1.5027798456972452
Sharpe Ratio for Soybeans Jul: -1.4588263806585897
Sharpe Ratio for Soybeans Sep: -1.3478167163800001
Sharpe Ratio for Wheat Mar: -0.8546812691828882
Sharpe Ratio for Wheat May: -0.8916929018523267
Sharpe Ratio for Wheat Jul: -0.9786382504443876
Sharpe Ratio for Wheat Sep: -1.0087996358251485


Negative Sharpe Ratio shows that it is unwise to invest in any of these stocks.

## Sortino Ratio

Similar to Sharpe, with different normalization.

In [10]:
for key in sharpe_dfs:
    sortino_ratio = sharpe_dfs[key]['Excess Return'].mean() / sharpe_dfs[key]['Excess Return'][sharpe_dfs[key]['Excess Return'] < 0].std() * np.sqrt(len(sharpe_dfs[key]))
    print(f'Sortino Ratio for {key}: {sortino_ratio}')


Sortino Ratio for Maize Mar: -2.3979569244289363
Sortino Ratio for Maize May: -2.3705700568236425
Sortino Ratio for Maize Jul: -2.3393139577126525
Sortino Ratio for Maize Sep: -2.215687062453782
Sortino Ratio for Oats Mar: -1.4931529406880437
Sortino Ratio for Oats May: -1.584773981079656
Sortino Ratio for Oats Jul: -1.3795123867029842
Sortino Ratio for Oats Sep: -1.4112165873647535
Sortino Ratio for Soybeans Mar: -2.1627456810112187
Sortino Ratio for Soybeans May: -1.9892199012523053
Sortino Ratio for Soybeans Jul: -1.898851423916218
Sortino Ratio for Soybeans Sep: -1.8208290925719466
Sortino Ratio for Wheat Mar: -1.5935713710136574
Sortino Ratio for Wheat May: -1.6499647919438283
Sortino Ratio for Wheat Jul: -1.818541609483092
Sortino Ratio for Wheat Sep: -1.8948227453162838


## Beta

Computing the beta of each stock

In [20]:
market = yf.Ticker('^GSPC')
market = market.history(period='5y')
market.reset_index(inplace=True)
market['Date'] = market['Date'].dt.date
market = market[['Date', 'Close']]
market['Daily Return'] = market['Close'].pct_change()
market.dropna(inplace=True)

In [21]:
market

,Date,Close,Daily Return
1,2019-11-01,3066.909912,0.009662
2,2019-11-04,3078.270020,0.003704
3,2019-11-05,3074.620117,-0.001186
4,2019-11-06,3076.780029,0.000702
5,2019-11-07,3085.179932,0.002730
...,...,...,...
1254,2024-10-25,5808.120117,-0.000299
1255,2024-10-28,5823.520020,0.002651
1256,2024-10-29,5832.919922,0.001614
1257,2024-10-30,5813.669922,-0.003300


## Treynor Ratio / Beta

The beta statistic of a stock shows its behavior compared to the behavior of the market (such as S&P 500).

In [ ]:
# calculate treynor ratio
market_return = stock_information['Prime'].iloc[-252:].mean()
for key in separate_dfs:
    beta = np.cov(separate_dfs[key]['Daily Return'], risk_free_stock['Daily Return'])[0][1] / np.var(risk_free_stock['Daily Return'])
    treynor_ratio = (separate_dfs[key]['Daily Return'].mean() - risk_free_stock['Daily Return'].mean()) / beta
    print(f'Treynor Ratio for {key}: {treynor_ratio}')

In [24]:
treynor = {}
treynor['beta'] = {}
treynor['treynor_ratio'] = {}

market_returns = market['Daily Return']

for key in sharpe_dfs:
    # Prepare the data for excess returns
    returns = sharpe_dfs[key]['Daily Return_x']
    treynor[key] = pd.concat([returns, market_returns], axis=1)
    
    treynor[key].columns = ['Stock Return', 'Market Return']
    treynor[key].dropna(inplace=True)

    # Calculate beta using covariance
    covariance = np.cov(treynor[key]['Stock Return'], treynor[key]['Market Return'])[0][1]
    market_variance = np.var(treynor[key]['Market Return'])
    beta = covariance / market_variance if market_variance != 0 else np.nan

    print(f'Beta for {key}: {beta}')
    
    


Beta for Maize Mar: 0.004876773482069927
Beta for Maize May: 0.005851610998317106
Beta for Maize Jul: 0.013162220421000154
Beta for Maize Sep: 0.018586223786738812
Beta for Oats Mar: 0.10167335898272592
Beta for Oats May: 0.08739106470103565
Beta for Oats Jul: 0.10178055510491545
Beta for Oats Sep: 0.03229574920780873
Beta for Soybeans Mar: 0.03338409980186514
Beta for Soybeans May: 0.0328857117337672
Beta for Soybeans Jul: 0.030276016681009776
Beta for Soybeans Sep: 0.02724707282433208
Beta for Wheat Mar: -0.017314063606265574
Beta for Wheat May: -0.018451479842896384
Beta for Wheat Jul: -0.017875161120417007
Beta for Wheat Sep: -0.013622547328367348


The beta values are very low ($\approx 0$), making the Treynor Ratio not informative.

## Jensen Alpha

The Jensen Alpha measures the